Получите данные по энергопотреблению первых 20 зданий (building_id от 0 до 19).

Заполните отсутствующие значения по погоде интерполяционными данными.

Разделите данные на обучающие/проверочные в пропорции 80/20.

Постройте и найдите общее качество модели линейной регрессии, построенной по часам для каждого из первых 20 зданий по следующим параметрам: 
* air_temperature, 
* dew_temperature,
* cloud_coverage,
* wind_speed,
* precip_depth_1_hr,
* sea_level_pressure,
* is_holiday.

Всего требуется построить 480 моделей линейной регрессии, вычислить по ним проверочные значения энергопотребления и получить итоговую оценку качества такой модели.
Для расчета последнего параметра (is_holiday) используйте график публичных выходных в США: USFederalHolidayCalendar

Исходные данные:
 * video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
 * video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
 * video.ittensive.com/machine-learning/ashrae/train.0.csv.gz

In [1]:
# библиотеки
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

загрузка данных

In [5]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy_ = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

In [6]:
# объединяем данные
energy_ = pd.merge(left=energy_, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
energy_ = energy_.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy_ = pd.merge(left=energy_, right=weather, how="left",
                  left_index=True, right_index=True)
energy_.reset_index(inplace=True)
energy_ = energy_[energy_["building_id"]<20] # отсекаем 20 зданий
energy_ = energy_.drop(columns=["meter", "site_id", "square_feet", "year_built", "floor_count"], axis=1)
del buildings # удаляем ненужные наборы данных освобождаем память
del weather # удаляем ненужные наборы данных освобождаем память
print (energy_.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 12059525
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   timestamp           175680 non-null  object 
 1   building_id         175680 non-null  int64  
 2   meter_reading       175680 non-null  float64
 3   primary_use         175680 non-null  object 
 4   air_temperature     175620 non-null  float64
 5   cloud_coverage      99080 non-null   float64
 6   dew_temperature     175620 non-null  float64
 7   precip_depth_1_hr   175660 non-null  float64
 8   sea_level_pressure  173980 non-null  float64
 9   wind_direction      170680 non-null  float64
 10  wind_speed          175680 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 16.1+ MB
None


оптимизируем память при помощи примеров функций предоставленными учебным центром ITtensive

In [7]:
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df

def round_fillna (df, columns):
    for col in columns:
        type_ = "int8"
        if col in ["wind_direction", "year_built", "precip_depth_1_hr"]:
            type_ = "int16"
        if col == "precip_depth_1_hr":
            df[col] = df[col].apply(lambda x:0 if x<0 else x)
        df[col] = np.round(df[col].fillna(value=0)).astype(type_)
    return df

energy_ = reduce_mem_usage(energy_)

Потребление памяти меньше на 10.39 Мб (минус 64.6 %)


In [8]:

print (energy_.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175680 entries, 0 to 12059525
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           175680 non-null  datetime64[ns]
 1   building_id         175680 non-null  int8          
 2   meter_reading       175680 non-null  float16       
 3   primary_use         175680 non-null  category      
 4   air_temperature     175620 non-null  float16       
 5   cloud_coverage      99080 non-null   float16       
 6   dew_temperature     175620 non-null  float16       
 7   precip_depth_1_hr   175660 non-null  float16       
 8   sea_level_pressure  173980 non-null  float16       
 9   wind_direction      170680 non-null  float16       
 10  wind_speed          175680 non-null  float16       
dtypes: category(1), datetime64[ns](1), float16(8), int8(1)
memory usage: 5.7 MB
None


дополнения данных о погоде интерполяцией

In [9]:
energy_["precip_depth_1_hr"] = energy_["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0) # 0 если кол-во осадков отрицательное
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "precip_depth_1_hr", "sea_level_pressure"]
# перебираем все колонки набора данных и расчитываем недостающие данные используя интерполяцию
for col in interpolate_columns: 
    energy_[col] = energy_[col].interpolate(limit_direction='both',
                            kind='cubic')

In [10]:
#добавляем в данные час
energy_["timestamp"] = pd.to_datetime(energy_["timestamp"]) # преобразуем поле в "датувремя"
energy_["hour"] = energy_["timestamp"].dt.hour.astype("int8") # бодавляем поле "час"
#добавляем праздники
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy_["date"] = pd.to_datetime(energy_["timestamp"].dt.date)
energy_['is_holiday'] = (energy_['date'].isin(us_holidays)).astype("int8")
print (energy_.head())

   timestamp  building_id  meter_reading primary_use  air_temperature  \
0 2016-01-01            0            0.0   Education             25.0   
1 2016-01-01            1            0.0   Education             25.0   
2 2016-01-01            2            0.0   Education             25.0   
3 2016-01-01            3            0.0   Education             25.0   
4 2016-01-01            4            0.0   Education             25.0   

   cloud_coverage  dew_temperature  precip_depth_1_hr  sea_level_pressure  \
0             6.0             20.0                0.0              1019.5   
1             6.0             20.0                0.0              1019.5   
2             6.0             20.0                0.0              1019.5   
3             6.0             20.0                0.0              1019.5   
4             6.0             20.0                0.0              1019.5   

   wind_direction  wind_speed  hour       date  is_holiday  
0             0.0         0.0     0 2

деллим данные 80/20

In [11]:
energy_train, energy_test = train_test_split(energy_[energy_["meter_reading"]>0], test_size=0.2)
print (energy_train.head())


                  timestamp  building_id  meter_reading primary_use  \
5283210 2016-06-11 07:00:00            7      491.75000   Education   
8718831 2016-09-22 15:00:00            5       62.46875   Education   
6831204 2016-07-27 19:00:00           11      466.75000   Education   
5078570 2016-06-05 04:00:00           13      291.75000   Education   
6632420 2016-07-21 18:00:00            8      388.50000   Education   

         air_temperature  cloud_coverage  dew_temperature  precip_depth_1_hr  \
5283210         25.00000         5.21875        22.203125                0.0   
8718831         29.40625         4.00000        22.796875                0.0   
6831204         34.40625         2.00000        20.000000                0.0   
5078570         26.09375         2.00000        22.796875                0.0   
6632420         30.59375         6.00000        22.203125                0.0   

         sea_level_pressure  wind_direction  wind_speed  hour       date  \
5283210         

готовим переменные для расчета регресии

In [21]:
# знаем что зданий 20, но правильней определить переменную через выбор максимального значения для последующего коректного
# исполнения кода в случае изменения условий анализа.
# print (energy_train["building_id"].max())

19


In [12]:
energy_learn_80 = pd.DataFrame(energy_train,
    columns=["meter_reading", "hour", "building_id", "air_temperature", "dew_temperature",
             "sea_level_pressure", "wind_speed", "cloud_coverage", "is_holiday"]) # df выбираем столбцы для расчета регресии 
hours = range(0, 24) # переменная кол- во часов
builds = range(0, energy_learn_80["building_id"].max() + 1) #переменная кол-во зданий


массив моделей линейной регрессии - по зданию и часу

In [13]:
energy_lr = [[]]*len(builds) #массив (колво списков)для линейной регресии
for bldng in builds:
    energy_lr[bldng] = [[]]*len(hours)# колво элементов с списке (по часам)
    energy_train_b = energy_learn_80[energy_learn_80["building_id"]==bldng] # выбираем здание по текущему ID
    for hour in hours: #перебираем значение по часам
        energy_train_bh = energy_train_b[energy_train_b["hour"]==hour] # берём значения (строку) по часу
        y = energy_train_bh["meter_reading"] # передаем (запоминаем) потребление -> У
        x = energy_train_bh.drop(labels=["meter_reading",
            "hour", "building_id"], axis=1) # в Х передаем данные только необходимый для расчета (удаляем не нужное)
        model = LinearRegression().fit(x, y) # расчитываем модель линейной регресии
        energy_lr[bldng][hour] = model.coef_ # передаем в модель расчитанные коэфиценты
        energy_lr[bldng][hour] = np.append(energy_lr[bldng][hour], model.intercept_) # добавляем в список расчитаные данные
print (energy_lr)
del energy_learn_80

[[array([ 5.46484375e+00,  1.48046875e+00, -1.33666992e-01, -4.02734375e+00,
       -7.40234375e-01, -1.41250000e+01,  2.28859555e+02]), array([ 6.72265625e+00,  8.95507812e-01, -6.93359375e-01, -2.99609375e+00,
       -1.10546875e+00,  4.01562500e+00,  7.78818571e+02]), array([   5.6953125 ,    2.171875  ,    0.49438477,   -0.91357422,
         -0.85058594,  -17.4375    , -434.50336022]), array([ 5.68359375e+00,  1.97851562e+00, -5.06835938e-01, -3.05468750e+00,
       -2.13867188e+00, -2.23125000e+01,  6.02808573e+02]), array([ 6.56640625e+00,  1.34765625e+00, -4.06005859e-01, -3.47070312e+00,
       -1.38867188e+00, -1.38906250e+01,  4.87668508e+02]), array([ 9.57031250e+00, -1.43164062e+00, -1.03271484e-01, -1.67480469e+00,
       -1.22851562e+00, -2.02031250e+01,  1.62642305e+02]), array([ 9.54687500e+00, -2.57031250e+00,  2.94036865e-02, -2.96875000e-01,
       -1.38964844e+00, -1.65781250e+01,  5.31536602e+01]), array([   8.3671875 ,   -0.63525391,    0.66064453,   -0.77880859,


используя функцию предоставленную учебным центром, расчитываем качество порсторенных моделей

In [15]:
# солбцы для расчета аналогичные со столбцами при формировании df для линейной регрессии
cols=["meter_reading", "hour", "building_id", "air_temperature", "dew_temperature",
             "sea_level_pressure", "wind_speed", "cloud_coverage", "is_holiday"]
def calculate_model (x):
    model = energy_lr[x.building_id][x.hour]
    lr = np.sum([x[col] * model[i] for i,col in enumerate(cols[3:])])
    lr += model[len(cols)-3]
    if lr < 0:
        lr = 0
    x["meter_reading_lr_q"] = (np.log(x.meter_reading + 1) -
                               np.log(1 + lr))**2
    return x

energy_test = energy_test.apply(calculate_model,
                                    axis=1, result_type="expand")
energy_test_lr_rmsle = np.sqrt(energy_test["meter_reading_lr_q"].sum() / len(energy_test))
print ("Качество линейной регрессии, 20 зданий:",
       energy_test_lr_rmsle, round(energy_test_lr_rmsle, 1))

Качество линейной регрессии, 20 зданий: 0.25371133421371284 0.3
